<a href="https://colab.research.google.com/github/knattarina/wildfire_challenge/blob/main/Weather_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##### IMPORTS AND SETUP #####

import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

import pandas as pd
import numpy as np
import datetime
import os
#import ibm_boto3

#from project_lib import Project
#from pyspark.sql import SparkSession
from google.colab import drive

#project = Project(sc,"ff75f9c0-4f85-495d-bbc1-6305f6b1dbb8", "p-12f1b4bbd0ab3b00eacd4ce1201f20ce039b72ec")
#spark = SparkSession.builder.getOrCreate()

In [8]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Wildfire_Challenge/Data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
##### IMPORT DATA #####

forecasts_df = pd.read_csv('Cleansed_Forecasts.csv')
aggr_df = pd.read_csv('Cleansed_Data.csv')